In [1]:
def INCLUDE(target, ipynb, key):
    import json
    with open(ipynb) as file:
        data = json.load(file)
        for cell in data['cells']:
            lines = cell['source']
            if type(lines) == str:
                lines = lines.split('\n')
            line0 = (lines or [''])[0].strip()
            if cell['cell_type'] == 'code' and line0 == f'### {key} ###':
                code = '\n'.join(lines)
                exec(code, target)

ipython = get_ipython()
colab   = 'google.colab' in str(ipython)
print(ipython)

if colab:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd '/content/gdrive/My Drive/Colab Notebooks'

!ls "./"


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks
ttt-3.ipynb  ttt-tactics-play.ipynb  ttt-tactics-tf.ipynb
ttt-tactics  ttt-tactics-test.ipynb  ttt-tactics-train.ipynb


In [2]:
notebook = f'./ttt-tactics-tf.ipynb'

INCLUDE(globals(), notebook, 'HEAD')
TESTS = False
INCLUDE(globals(), notebook, 'GAME')
INCLUDE(globals(), notebook, 'DATA')
INCLUDE(globals(), notebook, 'MODEL')

ipynb = 'ttt-tactics-train'


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8857399248202407291
,
 name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8417010894804108476
physical_device_desc: "device: XLA_CPU device"
]
tensorflow: 2.3.0
DIR = ./ttt-tactics
<class '__main__.AlphaZeroModel'>


<module 'matplotlib.pyplot' from '/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py'>

In [3]:
model  = AlphaZeroModel()
models = sortedfiles(f'{DIR}/model-*.h5')
model.load(models[-1])
num = int(re.findall(r'\d+', models[-1])[0]) 
print(f'num = ', num)

./ttt-tactics/model-000.h5 Oct 06 2020 14:41:29
num =  0


In [ ]:
path    = f'{DIR}/data-*.tfrec'
files   = sortedfiles(path)
dataset = loadsamples(files, buffer_size=2048, batch_size=64, seed=None)
print(f'found {len(files)} files at {path}')

schedule = { 2:0.0001 }
def scheduler(epoch, lr): 
    if epoch in schedule: return schedule[epoch]  
    else:                 return lr

history = model.fit(
    x                   = dataset,
    epochs              = 10,
    verbose             = 1,
    callbacks           = [LearningRateScheduler(scheduler)],
    workers             = 10,
    use_multiprocessing = False)



Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
Do not call `graph_parents`.
found 10 files at ./ttt-tactics/data-*.tfrec
Epoch 1/10
4327/4327 [==============================] - 278s 64ms/step - loss: 2.5637 - value_loss: 0.7552 - policy_loss: 1.8084 - value_lr: 9.9997e-04
Epoch 2/10
4327/4327 [==============================] - 275s 63ms/step - loss: 2.3540 - value_loss: 0.7442 - policy_loss: 1.6098 - value_lr: 9.9997e-04
Epoch 3/10
1567/4327 [=========>....................] - ETA: 3:00 - loss: 2.3436 - value_loss: 0.7486 - policy_loss: 1.5949 - value_lr: 1.0000e-04

In [ ]:
model.save(f'{DIR}/model-{num+1:0>3d}.h5')

In [ ]:
history = history.history
print(history)

In [ ]:
plt.plot(history['loss'], label='total loss')
plt.plot(history['policy_loss'], label='policy loss')
plt.plot(history['value_loss'], label='value loss')
plt.title('training history')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(loc="upper left")
plt.show()


In [ ]:
model  = BabyAlphaZeroModel()
models = sortedfiles(f'{DIR}/model-*.h5')
model.load(models[-1])
num = int(re.findall(r'\d+', models[-1])[0]) 
print(f'num = ', num)

seed0 = int(100*rand())
print('seed0 =', seed0)

num += 1
err  = float('inf')
for i in range(sys.maxsize):
    seed(seed0+i)
    try:
        ss,aa,rr = loadsamples(DIR, skipzeros=False, limit=8)
    except Exception as e: 
        print(e)
        continue
    errs = model.train(ss,aa,rr,
            batch_size = 512,
            epochs     = 30,
            rates      = { 0:0.0001 },
            shuffle    = True)
    ok = errs[0] < err
    if ok:
        err  = errs[0]
        model.save(f'{DIR}/model-{num:0>3d}.h5')
    NOTIFY(f'{str(model)} trained, prev: {err}, curr: {errs}, ok: {ok}', title=ipynb)
    ss,aa,rr = None,None,None
    gc.collect()
        